In [1]:
import pandas as pd
import re

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
dutch_stopwords = stopwords.words('dutch')

In [2]:
# Read the preprocessed data
df = pd.read_csv('a:/df_cleaned.csv')
df.head()


df = pd.read_csv('a:/test-data/bloeding-met-patientenlijst-copy-4-verslagen.csv')
df['alltext'] = df['verslagen_report_content']
df['date'] = pd.to_datetime(df['verslagen_report_start_date'], errors='coerce')
df.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_age_at_time_of_event,verslagen_report_specialism,verslagen_report_start_date,alltext,date
0,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Telefonisch consult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,66,"Maag-, Darm- en Leverziekten",2019-11-25 12:45:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-11-25 12:45:00
1,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VDER-[LASTNAME]...,66,"Maag-, Darm- en Leverziekten",2019-11-08 00:00:00,PATIENT : [LASTNAME] VDER-[LASTNAME]...,2019-11-08 00:00:00
2,0001A243648CFB955C3718FD1CE81276D1F4F260,Poliklinische Brief,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",65,"Maag-, Darm- en Leverziekten",2019-05-09 15:45:00,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",2019-05-09 15:45:00
3,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,65,"Maag-, Darm- en Leverziekten",2019-05-09 15:38:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-05-09 15:38:00
4,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VAN DER-[LASTNA...,65,"Maag-, Darm- en Leverziekten",2019-03-26 00:00:00,PATIENT : [LASTNAME] VAN DER-[LASTNA...,2019-03-26 00:00:00


In [3]:
# Add a unique ID per report based on pseudo_id and a sequential number
df['report_id'] = df.groupby('pseudo_id').cumcount().astype(str).str.zfill(3)
df['report_id'] = df['pseudo_id'] + '_' + df['report_id']

# Display the updated dataframe
df.head()



,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_age_at_time_of_event,verslagen_report_specialism,verslagen_report_start_date,alltext,date,report_id
0,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Telefonisch consult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,66,"Maag-, Darm- en Leverziekten",2019-11-25 12:45:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-11-25 12:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_000
1,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VDER-[LASTNAME]...,66,"Maag-, Darm- en Leverziekten",2019-11-08 00:00:00,PATIENT : [LASTNAME] VDER-[LASTNAME]...,2019-11-08 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_001
2,0001A243648CFB955C3718FD1CE81276D1F4F260,Poliklinische Brief,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",65,"Maag-, Darm- en Leverziekten",2019-05-09 15:45:00,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",2019-05-09 15:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_002
3,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,65,"Maag-, Darm- en Leverziekten",2019-05-09 15:38:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-05-09 15:38:00,0001A243648CFB955C3718FD1CE81276D1F4F260_003
4,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VAN DER-[LASTNA...,65,"Maag-, Darm- en Leverziekten",2019-03-26 00:00:00,PATIENT : [LASTNAME] VAN DER-[LASTNA...,2019-03-26 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_004


In [4]:
# Define regex pattern

# terms = [
#     # r"maagdarmbloeding", r"gastro[-\s]?intestinale bloeding",
#     # r" gib ", r" agib ", r"gi\s*bloeding",

#     r"bloedbraken", r"bloed\s*braken", r"hematemesis", r"melena",
#     r"maagdarmbloeding", r"gastro[-\s]?intestinale bloeding"
#     r"rectaal\s*bloedverlie", r"rectaal\s*bloedverlies", 
#     r"vers\s*bloed\s*per\s*anum",
#     r"bloeding\s*uit\s*het\s*maag[-\s]?darmkanaal",
# ]


# terms = [
#     r"bloedbraken", r"bloed\s*braken", r"hematemesis", r"melena",
#     r"maagdarmbloeding", r"gastro[-\s]?intestinale bloeding",
#     r"rectaal\s*bloedverlies?",  # aangepast van bloedverlies naar bloedverlie
#     r"vers\s*bloed\s*per\s*anum",
#     r"bloeding\s*uit\s*het\s*maag[-\s]?darmkanaal",
#     r" gib ", r" agib ", r"gi\s*bloeding",
#     r"anemie"
# ]



# # Escape multi-word phrases properly
# pattern_str = "|".join([fr"\b{t}\b" if " " not in t and "[- ]" not in t else t for t in terms])
# agib_pattern = re.compile(pattern_str, re.IGNORECASE)

agib_terms = [
    r"bloedbraken", r"bloed\s*braken", r"hematemesis",
    r"maagdarmbloeding", r"gastro[-\s]?intestinale bloeding",
    r"rectaal\s*bloedverlies?", r"vers\s*bloed\s*per\s*anum",
    r"bloeding\s*uit\s*het\s*maag[-\s]?darmkanaal",
    r" gib ", r" agib ", r"gi\s*bloeding"
]
agib_pattern = re.compile("|".join(agib_terms), re.IGNORECASE)

# Melena + context
melena_core = r"\b(melena|zwarte\s*ontlasting)\b"
support_terms = r"\b(hematemesis|bloedbraken|dalend\s*hb|shock|endoscopie|transfusie|opname|maagdarmbloeding)\b"
melena_pattern = re.compile(melena_core + r".*?" + support_terms, re.IGNORECASE)

def is_agib(text):
    text = str(text)
    return bool(agib_pattern.search(text) or melena_pattern.search(text))

df['is_agib'] = df['alltext'].apply(is_agib)
print(df['is_agib'])


def extract_context(text, pattern, window=5):
    matches = []
    for match in pattern.finditer(text):
        start, end = match.start(), match.end()
        before = text[:start]
        after = text[end:]

        # Get words before match
        before_words = re.findall(r'\w+', before)[-window:]
        # Get words after match
        after_words = re.findall(r'\w+', after)[:window]

        context = ' '.join(before_words + [match.group()] + after_words)
        matches.append(context)

    return matches[0] if matches else None

# Apply function
df['agib_context'] = df['alltext'].apply(lambda x: extract_context(str(x), agib_pattern))

# Function to extract the exact match
def extract_match(text, pattern):
    match = pattern.search(text)
    return match.group() if match else None

# Apply function to extract the exact match
df['agib_match'] = df['alltext'].apply(lambda x: extract_match(str(x), agib_pattern))
df.head()



0         False
1         False
2         False
3         False
4         False
          ...  
127691    False
127692    False
127693    False
127694    False
127695    False
Name: is_agib, Length: 127696, dtype: bool


,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_age_at_time_of_event,verslagen_report_specialism,verslagen_report_start_date,alltext,date,report_id,is_agib,agib_context,agib_match
0,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Telefonisch consult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,66,"Maag-, Darm- en Leverziekten",2019-11-25 12:45:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-11-25 12:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_000,False,None,None
1,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VDER-[LASTNAME]...,66,"Maag-, Darm- en Leverziekten",2019-11-08 00:00:00,PATIENT : [LASTNAME] VDER-[LASTNAME]...,2019-11-08 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_001,False,None,None
2,0001A243648CFB955C3718FD1CE81276D1F4F260,Poliklinische Brief,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",65,"Maag-, Darm- en Leverziekten",2019-05-09 15:45:00,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",2019-05-09 15:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_002,False,None,None
3,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,65,"Maag-, Darm- en Leverziekten",2019-05-09 15:38:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-05-09 15:38:00,0001A243648CFB955C3718FD1CE81276D1F4F260_003,False,None,None
4,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VAN DER-[LASTNA...,65,"Maag-, Darm- en Leverziekten",2019-03-26 00:00:00,PATIENT : [LASTNAME] VAN DER-[LASTNA...,2019-03-26 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_004,False,None,None


In [5]:
# def extract_relevant_sections(text, relevant_headers=None):
#     if relevant_headers is None:
#         relevant_headers = ["anamnese", "conclusie", "beleid", "tractusanamnese", "triagekeuze"]

#     # Split op kopjes
#     sections = re.split(r"(?<=\n)([A-Z][a-z]+(?:\s*[A-Z][a-z]+)*):", text)
    
#     # Combine sections into dict: {header: content}
#     it = iter(sections)
#     section_dict = {}
#     current = next(it, None)
#     while current:
#         header = current.strip().lower()
#         content = next(it, "")
#         section_dict[header] = content.strip()
#         current = next(it, None)

#     # Extract relevant parts only
#     combined = ""
#     for header in relevant_headers:
#         if header in section_dict:
#             combined += section_dict[header] + "\n"
    
#     return combined


In [6]:
def extract_context(text, pattern, window=5):
    matches = []
    for match in pattern.finditer(text):
        start, end = match.start(), match.end()
        before = text[:start]
        after = text[end:]

        # Get words before match
        before_words = re.findall(r'\w+', before)[-window:]
        # Get words after match
        after_words = re.findall(r'\w+', after)[:window]

        context = ' '.join(before_words + [match.group()] + after_words)
        matches.append(context)

    return matches[0] if matches else None





In [7]:
# Apply regex
df['agib_mention'] = df['alltext'].apply(lambda x: bool(agib_pattern.search(str(x))))

# Optionally exclude negated matches
negation_pattern = re.compile(r"\b(geen|niet|zonder)\b.{0,20}?\b(bloed(braken)?|hematemesis|melena|maagdarmbloeding|gastro[- ]intestinale bloeding|rectaal bloedverlies|vers bloed per anum|bloeding uit het maag[- ]darmkanaal)\b", re.IGNORECASE)
df['negated'] = df['alltext'].apply(lambda x: bool(negation_pattern.search(str(x))))
df['agib_final'] = df['agib_mention'] & ~df['negated']


# Apply function
df['agib_context'] = df['alltext'].apply(lambda x: extract_context(str(x), agib_pattern))
df.head()
# Function to extract the exact match
def extract_match(text, pattern):
    match = pattern.search(text)
    return match.group() if match else None

# Apply function to extract the exact match
df['agib_match'] = df['alltext'].apply(lambda x: extract_match(str(x), agib_pattern))
df.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_age_at_time_of_event,verslagen_report_specialism,verslagen_report_start_date,alltext,date,report_id,is_agib,agib_context,agib_match,agib_mention,negated,agib_final
0,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Telefonisch consult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,66,"Maag-, Darm- en Leverziekten",2019-11-25 12:45:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-11-25 12:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_000,False,None,None,False,False,False
1,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VDER-[LASTNAME]...,66,"Maag-, Darm- en Leverziekten",2019-11-08 00:00:00,PATIENT : [LASTNAME] VDER-[LASTNAME]...,2019-11-08 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_001,False,None,None,False,False,False
2,0001A243648CFB955C3718FD1CE81276D1F4F260,Poliklinische Brief,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",65,"Maag-, Darm- en Leverziekten",2019-05-09 15:45:00,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",2019-05-09 15:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_002,False,None,None,False,False,False
3,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,65,"Maag-, Darm- en Leverziekten",2019-05-09 15:38:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-05-09 15:38:00,0001A243648CFB955C3718FD1CE81276D1F4F260_003,False,None,None,False,False,False
4,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VAN DER-[LASTNA...,65,"Maag-, Darm- en Leverziekten",2019-03-26 00:00:00,PATIENT : [LASTNAME] VAN DER-[LASTNA...,2019-03-26 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_004,False,None,None,False,False,False


In [8]:
# # Nieuwe kolom met alleen relevante tekst
# df['agib_relevant_text'] = df['alltext'].apply(lambda x: extract_relevant_sections(str(x)))

# # Match alleen in relevante tekst
# df['agib_mention_relevant'] = df['agib_relevant_text'].apply(lambda x: bool(agib_pattern.search(x)))

# # Context + match optioneel:
# df['agib_match'] = df['agib_relevant_text'].apply(lambda x: extract_match(str(x), agib_pattern))
# df['agib_context'] = df['agib_relevant_text'].apply(lambda x: extract_context(str(x), agib_pattern))


In [9]:
# # Apply regex
# df['agib_mention'] = df['alltext'].apply(lambda x: bool(agib_pattern.search(str(x))))

# # Optionally exclude negated matches
# negation_pattern = re.compile(r"\b(geen|niet|zonder)\b.{0,20}?\b(bloed(braken)?|hematemesis|melena|maagdarmbloeding|gastro[- ]intestinale bloeding|rectaal bloedverlies|vers bloed per anum|bloeding uit het maag[- ]darmkanaal)\b", re.IGNORECASE)
# df['negated'] = df['alltext'].apply(lambda x: bool(negation_pattern.search(str(x))))
# df['agib_final'] = df['agib_mention'] & ~df['negated']

In [10]:
def extract_context(text, pattern, window=5):
    matches = []
    for match in pattern.finditer(text):
        start, end = match.start(), match.end()
        before = text[:start]
        after = text[end:]

        # Get words before match
        before_words = re.findall(r'\w+', before)[-window:]
        # Get words after match
        after_words = re.findall(r'\w+', after)[:window]

        context = ' '.join(before_words + [match.group()] + after_words)
        matches.append(context)

    return matches[0] if matches else None



# Apply function
df['agib_context'] = df['alltext'].apply(lambda x: extract_context(str(x), agib_pattern))
df.head()
# Function to extract the exact match
def extract_match(text, pattern):
    match = pattern.search(text)
    return match.group() if match else None

# Apply function to extract the exact match
df['agib_match'] = df['alltext'].apply(lambda x: extract_match(str(x), agib_pattern))
df.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_age_at_time_of_event,verslagen_report_specialism,verslagen_report_start_date,alltext,date,report_id,is_agib,agib_context,agib_match,agib_mention,negated,agib_final
0,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Telefonisch consult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,66,"Maag-, Darm- en Leverziekten",2019-11-25 12:45:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-11-25 12:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_000,False,None,None,False,False,False
1,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VDER-[LASTNAME]...,66,"Maag-, Darm- en Leverziekten",2019-11-08 00:00:00,PATIENT : [LASTNAME] VDER-[LASTNAME]...,2019-11-08 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_001,False,None,None,False,False,False
2,0001A243648CFB955C3718FD1CE81276D1F4F260,Poliklinische Brief,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",65,"Maag-, Darm- en Leverziekten",2019-05-09 15:45:00,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",2019-05-09 15:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_002,False,None,None,False,False,False
3,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,65,"Maag-, Darm- en Leverziekten",2019-05-09 15:38:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-05-09 15:38:00,0001A243648CFB955C3718FD1CE81276D1F4F260_003,False,None,None,False,False,False
4,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VAN DER-[LASTNA...,65,"Maag-, Darm- en Leverziekten",2019-03-26 00:00:00,PATIENT : [LASTNAME] VAN DER-[LASTNA...,2019-03-26 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_004,False,None,None,False,False,False


In [11]:
print("Aantal agib_mention == True:", df['agib_mention'].sum())


Aantal agib_mention == True: 5114


In [12]:
print("Aantal agib_final == True:", df['agib_final'].sum())

Aantal agib_final == True: 4068


In [13]:
print("Aantal negated == True:", df['negated'].sum())

Aantal negated == True: 7702


In [14]:
# for i, row in df[df['agib_match'].notnull()].iterrows():
#     print(f"🩸 Date: {row['date']}")
#     print(f"🩸 Match: {row['agib_match']}")
#     print(f"📍 Context: {row['agib_context']}")
#     print(f"📜 Content: {row['verslagen_report_content']}")
#     print("-----")


In [15]:
# for i, row in df[df['pseudo_id'] == 'C71B153E9F184C29E8D39654E41C8B54586E3AF4'].iterrows():
#     print(f"🩸 Date: {row['date']}")
#     print(f"🩸 Match: {row['agib_match']}")
#     print(f"📍 Context: {row['agib_context']}")
#     print(f"📜 Content: {row['verslagen_report_content']}")
#     print("-----")


In [16]:
pseudo_ids_with_agib = df[df['agib_mention'] == True]['pseudo_id'].unique()
pseudo_ids_df = pd.DataFrame(pseudo_ids_with_agib, columns=['pseudo_id'])
pseudo_ids_df

,pseudo_id
0,0003C64B626E17E45309A443C70019E76AB0C407
1,00092AD0261B74F4A045C46F099C6639F180698D
2,0014B95412D70A7E16B3E429E767E6A7ACD2B415
3,00153BFBE5D128EA90653088EBED470277429395
4,001EA214DD839E9E30458FBCA0296513538B847F
...,...
711,EF04810F281C202634208F2091141C40EBD2BC3A
712,F5F86E4F57D98E65D464A42AC325615EFE0D5C31
713,F8C241E69061BB450778B7A4CC336FF520469A0A
714,F8D116F01EE0039678998F393FA337C10AD4F4E2


In [17]:
all_ids = df['pseudo_id'].unique()
all_ids_df = pd.DataFrame(all_ids, columns=['pseudo_id'])
all_ids_df

,pseudo_id
0,0001A243648CFB955C3718FD1CE81276D1F4F260
1,0003C64B626E17E45309A443C70019E76AB0C407
2,00067EE4483868D5C3EEA1E072648CA61959714D
3,0006C6C74E4AC9C249EBE1F33DAE8EFCCBFBA783
4,0007089F5A45B8DA493CB883F40336EF4CC3254D
...,...
3418,F797097BA39F81B05EEC3F251973743CA03CB9B4
3419,F8C241E69061BB450778B7A4CC336FF520469A0A
3420,F8D116F01EE0039678998F393FA337C10AD4F4E2
3421,FAA79717FF2C725767E9469350ACECF640E5FCBC


In [18]:
df_dbc = pd.read_csv('a:/test-data/bloeding-met-patientenlijst-copy-2-diagnose-db-cs.csv')
print(len(df_dbc)), df_dbc.head()

2522


(None,
                                   pseudo_id  \
 0  0021F09BD5BB8FEFE4BD37362731099019AA6EB5   
 1  0021F09BD5BB8FEFE4BD37362731099019AA6EB5   
 2  0021F09BD5BB8FEFE4BD37362731099019AA6EB5   
 3  002F7F499E7875932FB6F0DCE675B843AFB507A7   
 4  002F7F499E7875932FB6F0DCE675B843AFB507A7   
 
    diagnose_db_cs_dbc_specialty_code  \
 0                                318   
 1                                318   
 2                                318   
 3                                318   
 4                                318   
 
     diagnose_db_cs_dbc_specialty_code_description  \
 0  Gastro-enterologie (MDL, maag-darm-lever-arts)   
 1  Gastro-enterologie (MDL, maag-darm-lever-arts)   
 2  Gastro-enterologie (MDL, maag-darm-lever-arts)   
 3  Gastro-enterologie (MDL, maag-darm-lever-arts)   
 4  Gastro-enterologie (MDL, maag-darm-lever-arts)   
 
    diagnose_db_cs_dbc_diagnosis_code  \
 0                                612   
 1                                612   
 2    

In [19]:
# Merge df with df_dbc on pseudo_id
df = df.merge(df_dbc[['pseudo_id', 'diagnose_db_cs_dbc_diagnosis_code']], on='pseudo_id', how='left')

# Display the updated dataframe
df.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_age_at_time_of_event,verslagen_report_specialism,verslagen_report_start_date,alltext,date,report_id,is_agib,agib_context,agib_match,agib_mention,negated,agib_final,diagnose_db_cs_dbc_diagnosis_code
0,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Telefonisch consult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,66,"Maag-, Darm- en Leverziekten",2019-11-25 12:45:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-11-25 12:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_000,False,None,None,False,False,False,NaN
1,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VDER-[LASTNAME]...,66,"Maag-, Darm- en Leverziekten",2019-11-08 00:00:00,PATIENT : [LASTNAME] VDER-[LASTNAME]...,2019-11-08 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_001,False,None,None,False,False,False,NaN
2,0001A243648CFB955C3718FD1CE81276D1F4F260,Poliklinische Brief,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",65,"Maag-, Darm- en Leverziekten",2019-05-09 15:45:00,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",2019-05-09 15:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_002,False,None,None,False,False,False,NaN
3,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,65,"Maag-, Darm- en Leverziekten",2019-05-09 15:38:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-05-09 15:38:00,0001A243648CFB955C3718FD1CE81276D1F4F260_003,False,None,None,False,False,False,NaN
4,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VAN DER-[LASTNA...,65,"Maag-, Darm- en Leverziekten",2019-03-26 00:00:00,PATIENT : [LASTNAME] VAN DER-[LASTNA...,2019-03-26 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_004,False,None,None,False,False,False,NaN


In [20]:
# Filter rows where agib_final is True and check for missing diagnosis codes
missing_diagnosis = df[df['agib_final'] & df['diagnose_db_cs_dbc_diagnosis_code'].isnull()]

# Display the result
if missing_diagnosis.empty:
    print("All rows with agib_final == True have a diagnose_db_cs_dbc_diagnosis_code.")
else:
    print("There are rows with agib_final == True but no diagnose_db_cs_dbc_diagnosis_code:")
    print(missing_diagnosis)

There are rows with agib_final == True but no diagnose_db_cs_dbc_diagnosis_code:
                                       pseudo_id  \
140     0003C64B626E17E45309A443C70019E76AB0C407   
186     0003C64B626E17E45309A443C70019E76AB0C407   
532     00092AD0261B74F4A045C46F099C6639F180698D   
534     00092AD0261B74F4A045C46F099C6639F180698D   
748     00153BFBE5D128EA90653088EBED470277429395   
...                                          ...   
225726  275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E   
225731  275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E   
225733  275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E   
225739  275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E   
226401  277F7BDD60A4FD41E18390D8549DD3A009686F6B   

                       verslagen_report_tags  \
140                                  Consult   
186     Consult, Polikliniek: vervolgconsult   
532         Consult, Kliniek: vervolgconsult   
534         Consult, Kliniek: vervolgconsult   
748                      Poliklinische Brief   
...   

In [21]:
missing_diagnosis

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_age_at_time_of_event,verslagen_report_specialism,verslagen_report_start_date,alltext,date,report_id,is_agib,agib_context,agib_match,agib_mention,negated,agib_final,diagnose_db_cs_dbc_diagnosis_code
140,0003C64B626E17E45309A443C70019E76AB0C407,Consult,Samenvatting: \nAanvraag Endoscopie\r\n-Datum ...,26,"Maag-, Darm- en Leverziekten",2015-04-21 11:02:00,Samenvatting: \nAanvraag Endoscopie\r\n-Datum ...,2015-04-21 11:02:00,0003C64B626E17E45309A443C70019E76AB0C407_040,True,de dag sinds weekend helderood rectaal bloedve...,rectaal bloedverlies,True,False,True,NaN
186,0003C64B626E17E45309A443C70019E76AB0C407,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nVervolgconsult\r\n-Type regist...,23,"Maag-, Darm- en Leverziekten",2011-12-21 11:44:00,Samenvatting: \nVervolgconsult\r\n-Type regist...,2011-12-21 11:44:00,0003C64B626E17E45309A443C70019E76AB0C407_086,True,week wel veel klachten en rectaal bloedverlies...,rectaal bloedverlies,True,False,True,NaN
532,00092AD0261B74F4A045C46F099C6639F180698D,"Consult, Kliniek: vervolgconsult",Samenvatting: \nDecursus\r\n[ Melding patient ...,33,Interne Geneeskunde,2017-09-13 10:32:00,Samenvatting: \nDecursus\r\n[ Melding patient ...,2017-09-13 10:32:00,00092AD0261B74F4A045C46F099C6639F180698D_002,True,geen evidente dehydratie 2 helder rectaal bloe...,rectaal bloedverlies,True,False,True,NaN
534,00092AD0261B74F4A045C46F099C6639F180698D,"Consult, Kliniek: vervolgconsult",Samenvatting: \nDecursus\r\n-Type decursus: De...,33,Interne Geneeskunde,2017-09-13 04:59:00,Samenvatting: \nDecursus\r\n-Type decursus: De...,2017-09-13 04:59:00,00092AD0261B74F4A045C46F099C6639F180698D_004,True,normaal veel drinken 3 helder rectaal bloedver...,rectaal bloedverlies,True,False,True,NaN
748,00153BFBE5D128EA90653088EBED470277429395,Poliklinische Brief,"Mw. Y. Onderwater, huisarts\r\n[STREETNAME] VE...",47,"Maag-, Darm- en Leverziekten",2018-09-24 10:17:00,"Mw. Y. Onderwater, huisarts\r\n[STREETNAME] VE...",2018-09-24 10:17:00,00153BFBE5D128EA90653088EBED470277429395_000,True,dermatoloog voor Barron ligatie Samenvatting R...,Rectaal bloedverlies,True,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225726,275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nVervolgconsult\r\n-Type regist...,70,"Maag-, Darm- en Leverziekten",2016-10-03 16:16:00,Samenvatting: \nVervolgconsult\r\n-Type regist...,2016-10-03 16:16:00,275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E_028,True,Verwijzer Huisarts Naam Bolsius Verwijsreden R...,Rectaal bloedverlies,True,False,True,NaN
225731,275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nVervolgconsult\r\n-Type regist...,70,"Maag-, Darm- en Leverziekten",2016-08-17 09:36:00,Samenvatting: \nVervolgconsult\r\n-Type regist...,2016-08-17 09:36:00,275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E_033,True,Verwijzer Huisarts Naam Bolsius Verwijsreden R...,Rectaal bloedverlies,True,False,True,NaN
225733,275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nVervolgconsult\r\n-Type regist...,70,"Maag-, Darm- en Leverziekten",2016-08-10 10:31:00,Samenvatting: \nVervolgconsult\r\n-Type regist...,2016-08-10 10:31:00,275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E_035,True,Verwijzer Huisarts Naam Bolsius Verwijsreden R...,Rectaal bloedverlies,True,False,True,NaN
225739,275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E,"Consult, Polikliniek: eerste consult",Samenvatting: \nEerste consult\r\n-Type 1e con...,70,"Maag-, Darm- en Leverziekten",2016-07-13 08:44:00,Samenvatting: \nEerste consult\r\n-Type 1e con...,2016-07-13 08:44:00,275ED7C3636FE6E2A5286FCE5CF4C8D85E8A6B3E_041,True,Verwijzer Huisarts Naam Bolsius Verwijsreden R...,Rectaal bloedverlies,True,False,True,NaN


In [22]:
df

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_age_at_time_of_event,verslagen_report_specialism,verslagen_report_start_date,alltext,date,report_id,is_agib,agib_context,agib_match,agib_mention,negated,agib_final,diagnose_db_cs_dbc_diagnosis_code
0,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Telefonisch consult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,66,"Maag-, Darm- en Leverziekten",2019-11-25 12:45:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-11-25 12:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_000,False,None,None,False,False,False,NaN
1,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VDER-[LASTNAME]...,66,"Maag-, Darm- en Leverziekten",2019-11-08 00:00:00,PATIENT : [LASTNAME] VDER-[LASTNAME]...,2019-11-08 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_001,False,None,None,False,False,False,NaN
2,0001A243648CFB955C3718FD1CE81276D1F4F260,Poliklinische Brief,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",65,"Maag-, Darm- en Leverziekten",2019-05-09 15:45:00,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",2019-05-09 15:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_002,False,None,None,False,False,False,NaN
3,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,65,"Maag-, Darm- en Leverziekten",2019-05-09 15:38:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-05-09 15:38:00,0001A243648CFB955C3718FD1CE81276D1F4F260_003,False,None,None,False,False,False,NaN
4,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VAN DER-[LASTNA...,65,"Maag-, Darm- en Leverziekten",2019-03-26 00:00:00,PATIENT : [LASTNAME] VAN DER-[LASTNA...,2019-03-26 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_004,False,None,None,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277800,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nVerpleegkundige verslaglegging...,79,Spoedeisende Hulp,2010-11-10 20:29:00,Samenvatting: \nVerpleegkundige verslaglegging...,2010-11-10 20:29:00,FD8C682C1F4FDA1E5EC0B760D30875556419BD71_077,False,None,None,False,False,False,203.0
277801,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nVerpleegkundige verslaglegging...,79,Spoedeisende Hulp,2010-11-10 20:29:00,Samenvatting: \nVerpleegkundige verslaglegging...,2010-11-10 20:29:00,FD8C682C1F4FDA1E5EC0B760D30875556419BD71_077,False,None,None,False,False,False,203.0
277802,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nVerpleegkundige verslaglegging...,79,Spoedeisende Hulp,2010-11-10 20:29:00,Samenvatting: \nVerpleegkundige verslaglegging...,2010-11-10 20:29:00,FD8C682C1F4FDA1E5EC0B760D30875556419BD71_077,False,None,None,False,False,False,203.0
277803,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,"Consult, SEH",Samenvatting: \nVerpleegkundige verslaglegging...,79,Spoedeisende Hulp,2010-11-10 20:29:00,Samenvatting: \nVerpleegkundige verslaglegging...,2010-11-10 20:29:00,FD8C682C1F4FDA1E5EC0B760D30875556419BD71_077,False,None,None,False,False,False,203.0


In [23]:
# Read the preprocessed data
df_agib_patients = pd.read_csv('a:/df_cleaned.csv')
df_agib_patients.head()

,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_start_date,date,alltext,tokens
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Klinische Brief,"Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",2020-11-26 15:06:00,2020-11-26 15:06:00,dhr aj dingemans huisarts streetnaam city datu...,"['dhr', 'aj', 'dingemans', 'huisarts', 'street..."
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies obv diver...,2020-11-26 09:53:00,2020-11-26 09:53:00,samenvatting rectaal bloedverlie obvn divertik...,"['samenvatting', 'rectaal', 'bloedverlie', 'ob..."
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,2020-11-25 14:13:00,2020-11-25 14:13:00,coloscopie betreffen mw initials lastname adre...,"['coloscopie', 'betreffen', 'mw', 'initials', ..."
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Poliklinische Brief,GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,2020-11-25 13:48:00,2020-11-25 13:48:00,gastroscopie betreffen mw initials lastname ad...,"['gastroscopie', 'betreffen', 'mw', 'initials'..."
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,"Consult, Kliniek: vervolgconsult",Samenvatting: \nRectaal bloedverlies ; eenmali...,2020-11-25 08:47:00,2020-11-25 08:47:00,samenvatting rectaal bloedverlie eenmalig hd h...,"['samenvatting', 'rectaal', 'bloedverlie', 'ee..."


In [34]:
# Filter df to include only rows with pseudo_id present in df_agib_patients
ids = df_agib_patients['pseudo_id'].unique()
ids_to_remove = ids.tolist()

df_filtered = df[~df['pseudo_id'].isin(ids_to_remove)]
df_filtered
df_filtered.sort_values(by=['pseudo_id', 'date'], ascending=[True, False])
df_filtered


,pseudo_id,verslagen_report_tags,verslagen_report_content,verslagen_report_age_at_time_of_event,verslagen_report_specialism,verslagen_report_start_date,alltext,date,report_id,is_agib,agib_context,agib_match,agib_mention,negated,agib_final,diagnose_db_cs_dbc_diagnosis_code
0,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Telefonisch consult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,66,"Maag-, Darm- en Leverziekten",2019-11-25 12:45:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-11-25 12:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_000,False,None,None,False,False,False,NaN
1,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VDER-[LASTNAME]...,66,"Maag-, Darm- en Leverziekten",2019-11-08 00:00:00,PATIENT : [LASTNAME] VDER-[LASTNAME]...,2019-11-08 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_001,False,None,None,False,False,False,NaN
2,0001A243648CFB955C3718FD1CE81276D1F4F260,Poliklinische Brief,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",65,"Maag-, Darm- en Leverziekten",2019-05-09 15:45:00,"Dhr. A.P. Nederstigt, huisarts\r\nBerg 22 B\r\...",2019-05-09 15:45:00,0001A243648CFB955C3718FD1CE81276D1F4F260_002,False,None,None,False,False,False,NaN
3,0001A243648CFB955C3718FD1CE81276D1F4F260,"Consult, Polikliniek: vervolgconsult",Samenvatting: \nChronische HBV\nMaart 2018 sec...,65,"Maag-, Darm- en Leverziekten",2019-05-09 15:38:00,Samenvatting: \nChronische HBV\nMaart 2018 sec...,2019-05-09 15:38:00,0001A243648CFB955C3718FD1CE81276D1F4F260_003,False,None,None,False,False,False,NaN
4,0001A243648CFB955C3718FD1CE81276D1F4F260,Microbiologieverslag,PATIENT : [LASTNAME] VAN DER-[LASTNA...,65,"Maag-, Darm- en Leverziekten",2019-03-26 00:00:00,PATIENT : [LASTNAME] VAN DER-[LASTNA...,2019-03-26 00:00:00,0001A243648CFB955C3718FD1CE81276D1F4F260_004,False,None,None,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226617,2792FD06CD87E275A2A9FE783C85A1E6724A68FE,Zorgportaal Nee,"\r\n\r\n\r\nMáxima Medisch Centrum,\r\nint...",60,Interne Geneeskunde,2010-11-01 10:41:00,"\r\n\r\n\r\nMáxima Medisch Centrum,\r\nint...",2010-11-01 10:41:00,2792FD06CD87E275A2A9FE783C85A1E6724A68FE_167,False,None,None,False,False,False,NaN
226618,2792FD06CD87E275A2A9FE783C85A1E6724A68FE,Zorgportaal Nee,"\r\n\r\n\r\nMáxima Medisch Centrum,\r\nloc...",58,Interne Geneeskunde,2008-09-30 15:05:00,"\r\n\r\n\r\nMáxima Medisch Centrum,\r\nloc...",2008-09-30 15:05:00,2792FD06CD87E275A2A9FE783C85A1E6724A68FE_168,False,None,None,False,False,False,NaN
226619,2792FD06CD87E275A2A9FE783C85A1E6724A68FE,Microbiologieverslag,PATIENT : [LASTNAME]*JPFC [BIRTHDATE...,56,Interne Geneeskunde,2007-01-04 00:00:00,PATIENT : [LASTNAME]*JPFC [BIRTHDATE...,2007-01-04 00:00:00,2792FD06CD87E275A2A9FE783C85A1E6724A68FE_169,False,None,None,False,False,False,NaN
226620,2792FD06CD87E275A2A9FE783C85A1E6724A68FE,Zorgportaal Nee,Op 18-08-2005 zag ik dhr. [INITIALS] [LASTNAME...,55,"Maag-, Darm- en Leverziekten",2005-08-18 15:40:00,Op 18-08-2005 zag ik dhr. [INITIALS] [LASTNAME...,2005-08-18 15:40:00,2792FD06CD87E275A2A9FE783C85A1E6724A68FE_170,False,None,None,False,False,False,NaN


In [ ]:
# for i, row in df_filtered[df_filtered['is_agib'].notnull()].iterrows():
for i, row in df_filtered[df_filtered['is_agib'] == True].iterrows():
    print('test:', i)
    print(f"🩸 Report ID: {row['report_id']}")
    print(f"📜 DBC code: {row['diagnose_db_cs_dbc_diagnosis_code']}")
    print(f"🩸 Date: {row['date']}")
    print(f"🩸 Match: {row['agib_match']}")
    print(f"🩸 Match: {row['is_agib']}")
    print(f"📍 Context: {row['agib_context']}")
    print(f"📜 Content: {row['verslagen_report_content']}")
    print("-----")


In [29]:
specific_id = df_filtered[df_filtered['pseudo_id'] == '21031C9814F3BD6EFACD9FBC5AF409820CDABAB4']
specific_id['pseudo_id'].unique()   

array([], dtype=object)

In [28]:
for i, row in specific_id[specific_id['is_agib'] == True].iterrows():
    print('test:', i)
    print(f"🩸 Report ID: {row['report_id']}")
    print(f"🩸 Date: {row['date']}")
    print(f"🩸 Match: {row['agib_match']}")
    print(f"🩸 Match: {row['is_agib']}")
    print(f"📍 Context: {row['agib_context']}")
    print(f"📜 Content: {row['verslagen_report_content']}")
    print("-----")

